# Training Log Visualization

This notebook provides tools to visualize training loss and accuracy from log files. It can parse training logs and create comprehensive plots showing training progress over time.

## Features
- Parse training logs to extract step, loss, and accuracy data
- Create dual plots for loss and accuracy trends
- Add linear trend lines with slope information
- Display comprehensive training statistics
- Save plots to files or display interactively

## Import Required Libraries

In [ ]:
import re
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

# Set matplotlib to display plots inline
%matplotlib inline

# Set figure size default
plt.rcParams['figure.figsize'] = (12, 10)

## Configuration

Set the path to your log file here:

In [ ]:
# Configure log file path
LOG_FILE_PATH = "/tmp/logs.txt"

# Optional: Set save path for plots (leave None to display only)
SAVE_PATH = None  # e.g., "training_progress.png"

## Log Parsing Function

This function parses the training log file to extract step numbers, loss values, and accuracy values.

In [ ]:
def parse_log_file(log_path):
    """
    Parse the training log file to extract step, loss, and accuracy data.

    Args:
        log_path (str): Path to the log file

    Returns:
        tuple: (steps, losses, accuracies) as lists
    """
    steps = []
    losses = []
    accuracies = []

    # Regular expression to match the training step lines
    pattern = (
        r"step: (\d+)/\d+, loss: ([\d.]+), "
        r"tokens_per_sec: [\d.]+, accuracy: ([\d.]+)"
    )

    try:
        with open(log_path, "r") as file:
            for line in file:
                match = re.search(pattern, line)
                if match:
                    step = int(match.group(1))
                    loss = float(match.group(2))
                    accuracy = float(match.group(3))

                    steps.append(step)
                    losses.append(loss)
                    accuracies.append(accuracy)

    except FileNotFoundError:
        print(f"Error: Log file '{log_path}' not found.")
        return [], [], []
    except Exception as e:
        print(f"Error reading log file: {e}")
        return [], [], []

    return steps, losses, accuracies

## Plotting Function

This function creates matplotlib plots for loss and accuracy over training steps with trend lines.

In [ ]:
def create_plots(steps, losses, accuracies, save_path=None):
    """
    Create matplotlib plots for loss and accuracy over training steps.

    Args:
        steps (list): Training step numbers
        losses (list): Loss values
        accuracies (list): Accuracy values
        save_path (str, optional): Path to save the plot image
    """
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    fig.suptitle("Training Progress", fontsize=16, fontweight="bold")

    # Plot loss
    ax1.plot(steps, losses, "b-", linewidth=1.5, alpha=0.8)
    ax1.set_xlabel("Training Step")
    ax1.set_ylabel("Loss")
    ax1.set_title("Training Loss Over Time")
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, max(steps))

    # Add trend line for loss
    if len(steps) > 1:
        z = np.polyfit(steps, losses, 1)
        p = np.poly1d(z)
        ax1.plot(
            steps,
            p(steps),
            "r--",
            alpha=0.8,
            linewidth=1,
            label=f"Trend (slope: {z[0]:.6f})",
        )
        ax1.legend()

    # Plot accuracy
    ax2.plot(steps, accuracies, "g-", linewidth=1.5, alpha=0.8)
    ax2.set_xlabel("Training Step")
    ax2.set_ylabel("Accuracy")
    ax2.set_title("Training Accuracy Over Time")
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(0, max(steps))
    ax2.set_ylim(0, max(max(accuracies) * 1.1, 1.0))

    # Add trend line for accuracy
    if len(steps) > 1:
        z = np.polyfit(steps, accuracies, 1)
        p = np.poly1d(z)
        ax2.plot(
            steps,
            p(steps),
            "r--",
            alpha=0.8,
            linewidth=1,
            label=f"Trend (slope: {z[0]:.6f})",
        )
        ax2.legend()

    # Adjust layout
    plt.tight_layout()

    # Save or show plot
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"Plot saved to: {save_path}")

    plt.show()

## Statistics Function

This function prints comprehensive training statistics.

In [ ]:
def print_statistics(steps, losses, accuracies):
    """Print training statistics."""
    if not steps:
        print("No training data found in log file.")
        return

    print("\n" + "=" * 50)
    print("TRAINING STATISTICS")
    print("=" * 50)
    print(f"Total training steps: {len(steps)}")
    print(f"Step range: {min(steps)} - {max(steps)}")
    print(f"Initial loss: {losses[0]:.4f}")
    print(f"Final loss: {losses[-1]:.4f}")
    best_loss_step = steps[losses.index(min(losses))]
    print(f"Best loss: {min(losses):.4f} (step {best_loss_step})")
    print(f"Initial accuracy: {accuracies[0]:.4f}")
    print(f"Final accuracy: {accuracies[-1]:.4f}")
    best_acc_step = steps[accuracies.index(max(accuracies))]
    print(f"Best accuracy: {max(accuracies):.4f} (step {best_acc_step})")
    print(f"Loss improvement: {losses[0] - losses[-1]:.4f}")
    print(f"Accuracy improvement: {accuracies[-1] - accuracies[0]:.4f}")
    print("=" * 50)

## Load and Parse Training Data

Now let's load the training log and extract the data:

In [ ]:
# Check if log file exists
if not Path(LOG_FILE_PATH).exists():
    print(f"Error: Log file '{LOG_FILE_PATH}' does not exist.")
    print("Please update the LOG_FILE_PATH variable in the configuration cell above.")
else:
    print(f"Parsing log file: {LOG_FILE_PATH}")

    # Parse the log file
    steps, losses, accuracies = parse_log_file(LOG_FILE_PATH)

    if not steps:
        print("No training data found in the log file.")
    else:
        print(f"Successfully parsed {len(steps)} training steps.")

## Display Training Statistics

Let's examine the training statistics:

In [ ]:
if 'steps' in locals() and steps:
    print_statistics(steps, losses, accuracies)
else:
    print("No training data available. Please run the data loading cell first.")

## Create Training Progress Plots

Now let's visualize the training progress:

In [ ]:
if 'steps' in locals() and steps:
    create_plots(steps, losses, accuracies, SAVE_PATH)
else:
    print("No training data available. Please run the data loading cell first.")